# Importações

In [1]:
#import os
#import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.metrics import classification_report
#from sklearn.datasets import load_digits
#from FirstStage import FirstStage
#from SecondStage import SecondStage
#from Extension import Extension

from tqdm import tqdm


# Dados

### Carregando os dados

In [2]:
cic_ids_2017 = pd.read_parquet("data/cic_ids_2017.parquet")
cse_cic_ids_2018 = pd.read_parquet("data/cse_cic_ids_2018.parquet")
#testp = pd.read_parquet("data/test.parquet")
#cic_collection = pd.read_parquet("data/cic-collection.parquet")
#infiltration_2018 = pd.read_parquet("data/infiltration_2018.parquet")

Exibindo o **.info()**

In [3]:
cic_ids_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             int64  
 1   Flow Duration                int64  
 2   Total Fwd Packets            int64  
 3   Total Backward Packets       int64  
 4   Total Length of Fwd Packets  int64  
 5   Total Length of Bwd Packets  int64  
 6   Fwd Packet Length Max        int64  
 7   Fwd Packet Length Min        int64  
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        int64  
 11  Bwd Packet Length Min        int64  
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 int64  
 19  

In [4]:
cse_cic_ids_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16233002 entries, 0 to 16233001
Data columns (total 78 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          float64
 1   Flow Duration      float64
 2   Tot Fwd Pkts       float64
 3   Tot Bwd Pkts       float64
 4   TotLen Fwd Pkts    float64
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    float64
 7   Fwd Pkt Len Min    float64
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    float64
 11  Bwd Pkt Len Min    float64
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       float64
 19  Flow IAT Min       float64
 20  Fwd IAT Tot        float64
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        float64
 24  Fwd IAT Min        float64
 25  Bwd IAT Tot     

### Padronizando e limpando os dados

#### Concatenando os dados

In [5]:
df = pd.concat([cic_ids_2017, cse_cic_ids_2018], axis=0)

In [6]:
# Limpeza, pra liberar memória
del cic_ids_2017
del cse_cic_ids_2018

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19063745 entries, 0 to 16233001
Columns: 129 entries, Destination Port to Fwd Seg Size Min
dtypes: float64(128), object(1)
memory usage: 18.5+ GB


#### Alterando Labels

In [8]:
pd.set_option('display.max_rows', 999)
values_count = df['Label'].value_counts()
print(values_count)

Label
Benign                        13484708
BENIGN                         2273097
DDOS attack-HOIC                686012
DDoS attacks-LOIC-HTTP          576191
DoS attacks-Hulk                461912
Bot                             288157
DoS Hulk                        231073
FTP-BruteForce                  193360
SSH-Bruteforce                  187589
Infilteration                   161934
PortScan                        158930
DoS attacks-SlowHTTPTest        139890
DDoS                            128027
DoS attacks-GoldenEye            41508
DoS attacks-Slowloris            10990
DoS GoldenEye                    10293
FTP-Patator                       7938
SSH-Patator                       5897
DoS slowloris                     5796
DoS Slowhttptest                  5499
DDOS attack-LOIC-UDP              1730
Web Attack � Brute Force          1507
Web Attack � XSS                   652
Brute Force -Web                   611
Brute Force -XSS                   230
SQL Injection      

Padronizando labels e corrigindo labels redundantes/repetidas ou com caracteres irreconhecíveis.

In [9]:
label_changes = {'infilteration':'Infiltration', 
                 'Infilteration':'Infiltration',  
                 'PortScan':'Port Scan', 
                 'Web Attack � Brute Force':'Brute Force-Web Attack', 
                 'Web Attack � XSS':'Web Attack-XSS', 
                 'Web Attack � Sql Injection':'Sql Injection-Web Attack', 
                 'DoS slowloris':'DoS Slowloris', 
                 'Benign':'BENIGN', 
                 'DoS Hulk':'DoS-Hulk', 
                 'DoS-Goldeneye':'DoS-GoldenEye', 
                 'DoS GoldenEye':'DoS-GoldenEye', 
                 'DoS Slowhttptest':'DoS-Slowhttptest', 
                 'DoS Slowloris':'DoS-Slowloris', 
                 'portscan':'Port Scan', 
                 'Webattack-bruteforce':'Brute Force-Web Attack', 
                 'Webattack-XSS':'Web Attack-XSS', 
                 'Webattack-SQLi':'Web Attack-SQLi', 
                 'DoS attacks-Hulk':'DoS-Hulk', 
                 'Brute Force -Web':'Brute Force-Web Attack', 
                 'DDoS attacks-LOIC-HTTP':'DDoS-LOIC-HTTP', 
                 'DDoS attack-HOIC':'DDoS-HOIC', 
                 'DDOS attack-LOIC-UDP':'DDOS-LOIC-UDP', 
                 'DoS attacks-SlowHTTPTest':'DoS-SlowHTTPTest', 
                 'DoS attacks-GoldenEye':'DoS-GoldenEye', 
                 'DoS attacks-Slowloris':'DoS-Slowloris', 
                 'SSH-Bruteforce':'Brute Force-SSH', 
                 'FTP-BruteForce':'Brute Force-FTP', 
                 'Brute Force -XSS':'Brute Force-XSS', 
                 'SSH-Patator':'Patator-SSH', 
                 'FTP-Patator':'Patator-FTP', 
                 'DDOS-LOIC-UDP':'DDoS-LOIC-UDP' }

df['Label'] = df['Label'].replace(label_changes)

Resultado:

In [10]:
pd.set_option('display.max_rows', None)
values_count = df['Label'].value_counts()
print(values_count.sort_values())

Label
Heartbleed                        11
Sql Injection-Web Attack          21
Label                             59
SQL Injection                     87
Brute Force-XSS                  230
Web Attack-XSS                   652
DDOS-LOIC-UDP                   1730
Brute Force-Web Attack          2118
DoS-Slowhttptest                5499
DoS Slowloris                   5796
Patator-SSH                     5897
Patator-FTP                     7938
DoS-Slowloris                  10990
DoS-GoldenEye                  51801
DDoS                          128027
DoS-SlowHTTPTest              139890
Port Scan                     158930
Infiltration                  161970
Brute Force-SSH               187589
Brute Force-FTP               193360
Bot                           288157
DDoS-LOIC-HTTP                576191
DDOS attack-HOIC              686012
DoS-Hulk                      692985
BENIGN                      15757805
Name: count, dtype: int64


#### Renomeando e removendo: linhas e colunas

Removendo os dados que tem a string **Label** na coluna **Label**. Não faço ideia do que seja isso.

In [11]:
df.drop(df[df['Label'] == 'Label'].index, inplace=True)

Removendo a coluna **Destination Port**, porque não existe muita correlação entre o tamanho do número da porta com a função da porta, por exemplo, não existe algo do tipo "portas maiores tem uma função mais pra X, enquanto portas menores servem mais pra Y".

In [12]:
df.drop(columns=['Destination Port'], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19063686 entries, 0 to 16233001
Columns: 128 entries, Flow Duration to Fwd Seg Size Min
dtypes: float64(127), object(1)
memory usage: 18.3+ GB


Existem algumas colunas que significam a mesma coisa, mas estão com nomes diferentes.

In [ ]:
df.rename(columns={'coloração': 'cor'}, inplace=True)

#### Registros duplicados

In [ ]:
pd.set_option('display.max_rows', 10) # Retornando a um valor menor na exibição do Pandas

In [ ]:
df[df.duplicated()]

Exibindo os registros não duplicados

In [ ]:
df[~df.duplicated()]

Descartando registros duplicados

In [ ]:
initial_len = df.shape[0]
df = df.drop_duplicates()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartadas {initial_len - df.shape[0]} duplicadas')

df = df.reset_index(drop=True) # Resetando index

#### Registros com valores não finitos

In [ ]:
df_columns_isfinite = np.isfinite(df.drop(['Label'], axis='columns')).all(axis=0)
df_columns_isfinite[df_columns_isfinite == False]

In [ ]:
df_rows_isfinite = np.isfinite(df.drop(['Label'], axis='columns')).all(axis=1)
inf_indexes = df_rows_isfinite[df_rows_isfinite == False].index
df.iloc[inf_indexes][['Flow Bytes/s', 'Flow Packets/s']]

Transformando valores infinitos no maior valor finito encontrado na coluna, isso é feito para as duas colunas.

Isso é feito porque a quantidade de registros infinitos é insignificante, seria inviável fazer isso em um conjunto de dados com muitos registros infinitos, pois existiriam muitos valores máximos e isso poderia comprometer o treinamento.

In [ ]:
max_finite_flow_packets_per_sec = df[np.isfinite(df['Flow Packets/s'])]['Flow Packets/s'].max()
max_finite_flow_bytes_per_sec = df[np.isfinite(df['Flow Bytes/s'])]['Flow Bytes/s'].max()

df.loc[df['Flow Packets/s'] == np.inf, 'Flow Packets/s'] = max_finite_flow_packets_per_sec
df.loc[df['Flow Bytes/s'] == np.inf, 'Flow Bytes/s'] = max_finite_flow_bytes_per_sec

df = df.reset_index(drop=True) # Resetando index

#### Registros com valores Null/NaN/NA

In [ ]:
df.columns[df.isna().any(axis=0)]

In [ ]:
df[df.isna().any(axis=1)][['Flow Bytes/s']]

Preenchendo registros NaN/Null/NA com a média dos valores de cada coluna.

In [ ]:
for column in tqdm(df.columns):
    if column != "Label":
        column_median = df[column].median()
        df[column].fillna(column_median, inplace=True)

df = df.reset_index(drop=True) # Resetando index

#### Features correlacionadas

In [ ]:
def get_highly_correlated_features(correlation_matrix, threshold):
  correlated_pairs = []
  for i in range(len(correlation_matrix.columns)):
    for j in range(i):
      if abs(correlation_matrix.iloc[i, j]) > threshold:
        pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
        coefficient = correlation_matrix.iloc[i, j]
        correlated_pairs.append((pair, coefficient))
  return sorted(correlated_pairs, key= lambda pair: pair[1], reverse=True)


Coletando as features correlacionadas, com o objetivo de evitar a redundância no treinamento do modelo.

In [ ]:
df_without_Label = df.drop('Label', axis='columns')

In [ ]:
corr_matrix = df_without_Label.corr().abs()
correlation_list = get_highly_correlated_features(corr_matrix, 0.95)

# Limpeza
del df_without_Label

In [ ]:
correlation_list[:10]

Criando uma lista do que será dropado.

In [ ]:
f2drop = []
for feature_pair, _ in correlation_list:
  if feature_pair[0] not in f2drop and feature_pair[1] not in f2drop:
    f2drop.append(feature_pair[1])

Exibindo as features correlacionadas.

In [ ]:
f2drop

Removendo as features que atrapalham o modelo de aprendizagem

In [ ]:
df = df.drop(f2drop, axis='columns')

#### Normalização dos dados

In [ ]:
std_scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = std_scaler.fit_transform(df[numeric_cols])
#df = pd.DataFrame(std_scaler.fit_transform(df), columns=df.columns)

#### Limpeza de variáveis

In [ ]:
del f2drop
del corr_matrix
del correlation_list
del df_columns_isfinite
del df_rows_isfinite
del inf_indexes
del max_finite_flow_packets_per_sec
del max_finite_flow_bytes_per_sec
del initial_len
del numeric_cols
del std_scaler
del label_changes
del values_count

### Dados após o tratamento

Exibindo o **.describe()**.

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
df.describe(include='all')

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

Exibindo o **.info()**.

In [ ]:
df.info()

# Dividindo os dados (treino, validação, teste)

In [ ]:
df_train = df.query('Label == "BENIGN"').sample(frac=0.6)#, random_state=RANDOM_SEED)
df_val_test = df.drop(df_train.index)

df_train = df_train.reset_index(drop=True)
df_val_test = df_val_test.reset_index(drop=True)

X_train = df_train.drop('Label', axis='columns')

In [ ]:
del df

In [ ]:
X_val, X_test, classes_val, classes_test = train_test_split(df_val_test.drop('Label', axis='columns'), df_val_test['Label'], test_size=0.65, stratify=df_val_test['Label']) #random_state=RANDOM_SEED)

X_val, X_test = X_val.reset_index(drop=True), X_test.reset_index(drop=True)
classes_val, classes_test =  classes_val.reset_index(drop=True), classes_test.reset_index(drop=True)

y_val, y_test = classes_val.apply(lambda c: 0 if c == 'BENIGN' else 1), classes_test.apply(lambda c: 0 if c == 'BENIGN' else 1)

In [ ]:
del df_train, df_val_test

# Ignorar

In [ ]:
# Valor específico
#valor_especifico = 'BENIGN'

# Coleta de valores diferentes do valor específico
#valores_diferentes = df.loc[df['Label'] != valor_especifico, 'Label'].tolist()

#print(valores_diferentes)


In [ ]:
#pd.set_option('display.max_rows', None)
#contagem_valores = df['Label'].value_counts()
#print(contagem_valores)


In [ ]:
#df['Z'] = df['X'] + df['Y']

# Apagar as colunas X e Y
#df.drop(columns=['X', 'Y'], inplace=True)